In [ ]:
# from google.colab import drive

# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
path_to_dataset = ''

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupShuffleSplit

In [8]:
load_data = True

In [9]:
input_file_train = open(path_to_dataset + 'train_dev/bn-train.conll', 'r', encoding='utf-8')
input_file_dev = open(path_to_dataset + 'train_dev/bn-dev.conll', 'r', encoding='utf-8')

raw_data_train = input_file_train.read()
raw_data_dev = input_file_dev.read()

## Reading the Data, formatting it and Saving it in CSV

In [7]:
if not load_data:
  input_file_train = open(path_to_dataset + 'train_dev/bn-train.conll', 'r', encoding='utf-8')
  input_file_dev = open(path_to_dataset + 'train_dev/bn-dev.conll', 'r', encoding='utf-8')

  raw_data_train = input_file_train.read()
  raw_data_dev = input_file_dev.read()

  def get_df_from_raw(raw):
    columns = ['sentence_id', 'words', 'labels']
    raw_list = raw.split('\n')
    output = []
    sent_id = -1
    for item in raw_list:
      if len(item) == 0:
        continue
      if item[0] == '#':
        sent_id += 1
        continue
      items = item.split(' _ _ ')
      row = [sent_id, items[0], items[1]]
      output.append(row)
    df = pd.DataFrame(output, columns = columns)
    return df

  ## Getting dataframe from Raw data
  data_train = get_df_from_raw(raw_data_train)
  data_dev = get_df_from_raw(raw_data_dev)

  #Checking the Data
  data_train.head(30)

  ### Saving the data for later use
  data_train.to_csv(path_to_dataset + 'data_train.csv', encoding='utf-8',index = False)
  data_dev.to_csv(path_to_dataset + 'data_dev.csv', encoding='utf-8', index = False)

## Reading from saved CSV

In [10]:
if load_data:
  ### Reading data from csv
  data_train_unsplitted = pd.read_csv(path_to_dataset + 'data_train.csv', encoding='utf-8')
  data_dev = pd.read_csv(path_to_dataset + 'data_dev.csv', encoding='utf-8')

  splitter = GroupShuffleSplit(test_size = .20, n_splits = 1, random_state = 7)
  split = splitter.split(data_train_unsplitted, groups=data_train_unsplitted['sentence_id'])
  train_inds, test_inds = next(split)

  data_train = data_train_unsplitted.iloc[train_inds]
  data_valid = data_train_unsplitted.iloc[test_inds]


In [11]:
data_train.head(50)

,sentence_id,words,labels
6,1,উদ্ভিদটির,O
7,1,৯০,O
8,1,০০০,O
9,1,এরও,O
10,1,বেশি,O
11,1,সৌর,B-OtherPROD
12,1,প্যানেল,I-OtherPROD
13,1,২৩৫,O
14,1,একরও,O
15,1,বেশি,O


In [ ]:
# X_train = data_train[['sentence_id', 'words']]
# y_train = data_train['labels']

# X_dev = data_dev[['sentence_id', 'words']]
# y_dev = data_dev['labels']

## Using bnunicodenormalizer for word normalization

In [12]:
!pip install bnunicodenormalizer

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.1.6-py3-none-any.whl size=22785 sha256=e968c027f9aa8a29d1ae8e0887c45f4f1d19e10e31d1b04e03c9310ea45857da
  Stored in directory: c:\users\21241049\appdata\local\pip\cache\wheels\e9\d8\cc\b2809f6116dc25b5b0e888b673eac1363b50888004ca43925a
Successfully built bnunicodenormalizer


In [13]:
from bnunicodenormalizer import Normalizer
from pprint import pprint

In [14]:
## using the normalizer
bnorm=Normalizer()
def MrNormalizer(word):
  result = bnorm(word)
  return result['normalized']
data_train['words'] = data_train['words'].map(MrNormalizer)
data_valid['words'] = data_valid['words'].map(MrNormalizer)

C:\Users\21241049\AppData\Local\Temp\ipykernel_19240\1631904719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['words'] = data_train['words'].map(MrNormalizer)
C:\Users\21241049\AppData\Local\Temp\ipykernel_19240\1631904719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_valid['words'] = data_valid['words'].map(MrNormalizer)


In [15]:
data_train

,sentence_id,words,labels
6,1,উদ্ভিদটির,O
7,1,৯০,O
8,1,০০০,O
9,1,এরও,O
10,1,বেশি,O
...,...,...,...
130166,9706,বেল,B-PrivateCorp
130167,9706,ল্যাবস,I-PrivateCorp
130168,9706,এ,O
130169,9706,যোগদান,O


## NERmodel of simpletransformer (using mBERT)

In [ ]:
!pip uninstall torch

In [1]:
!pip install torch torchvision --pre -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB 435.7 kB/s eta 0:00:14
     ---------------------------------------- 0.1/5.7 MB 544.7 kB/s eta 0:00:11
      --------------------------------------- 0.1/5.7 MB 798.9 kB/s eta 0:00:08
     - -------------------------------------- 0.3/5.7 MB 1.3 MB/s eta 0:00:05
     --- ------------------------------------ 0.4/5.7 MB 1.8 MB/s eta 0:00:03
     ---- ----------------------------------- 0.6/5.7 MB 2.2 MB/s eta 0:00:03
     ----- ---------------------------------- 0.8/5.7 MB 2.4 MB/s eta 0:00:03
     --------- -----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.16.4 requires fsspec, which is not installed.
huggingface-hub 0.16.4 requires tqdm>=4.42.1, which is not installed.
open-clip-torch 2.20.0 requires regex, which is not installed.
open-clip-torch 2.20.0 requires tqdm, which is not installed.
transformers 4.31.0 requires regex!=2019.12.17, which is not installed.
transformers 4.31.0 requires tqdm>=4.27, which is not installed.


In [2]:
import torch
print(torch.cuda.is_available())

False


In [16]:
!pip install simpletransformers

  Obtaining dependency information for simpletransformers from https://files.pythonhosted.org/packages/16/c8/20d7eede93e320c0746c01d2205bdbeb388c236247244b9428e743a96de9/simpletransformers-0.64.3-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.3 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 42.3/42.3 kB 509.2 kB/s eta 0:00:00
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/66/f8/38298237d18d4b6a8ee5dfe390e97bed5adb8e01ec6f9680c0ddf3066728/datasets-2.14.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for wandb>=0.10.32 from https://files.pythonhosted.org/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires clang~=5.0, which is not installed.
tensorflow 2.6.0 requires absl-py~=0.10, but you have absl-py 1.4.0 which is incompatible.
tensorflow 2.6.0 requires flatbuffers~=1.12, but you have flatbuffers 20210226132247 which is incompatible.


In [17]:
from simpletransformers.ner import NERModel, NERArgs

C:\Users\21241049\.conda\envs\cse431\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## Fetching all the labels provided in dataset
labels = data_train['labels'].unique().tolist()

In [ ]:
print("Number of Labels:", len(labels))
print(labels)

Number of Labels: 67
['O', 'B-OtherPROD', 'I-OtherPROD', 'B-HumanSettlement', 'B-ORG', 'I-ORG', 'I-HumanSettlement', 'B-PublicCorp', 'I-PublicCorp', 'B-Software', 'B-Medication/Vaccine', 'B-AerospaceManufacturer', 'I-AerospaceManufacturer', 'B-Disease', 'B-PrivateCorp', 'I-PrivateCorp', 'B-WrittenWork', 'I-WrittenWork', 'B-Artist', 'I-Artist', 'I-Software', 'B-OtherPER', 'B-Vehicle', 'I-Vehicle', 'I-Medication/Vaccine', 'B-Politician', 'I-Politician', 'B-Athlete', 'I-Athlete', 'I-OtherPER', 'B-VisualWork', 'I-VisualWork', 'B-Facility', 'B-CarManufacturer', 'I-CarManufacturer', 'B-Food', 'I-Facility', 'I-Disease', 'B-OtherLOC', 'I-OtherLOC', 'B-SportsManager', 'B-Scientist', 'B-Station', 'I-Station', 'B-Symptom', 'B-Cleric', 'B-SportsGRP', 'I-SportsGRP', 'I-Cleric', 'I-Scientist', 'B-ArtWork', 'B-MedicalProcedure', 'I-ArtWork', 'I-Food', 'B-Drink', 'I-SportsManager', 'I-MedicalProcedure', 'B-MusicalWork', 'I-MusicalWork', 'B-MusicalGRP', 'I-MusicalGRP', 'I-Symptom', 'B-AnatomicalStructu

In [ ]:
## Fine Tuning The Model
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32


In [ ]:
### Defining the model
model = NERModel('bert', 'bert-base-multilingual-cased', labels = labels, args =args)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## Counting the Null values
data_train.isna().sum()

sentence_id      0
words          472
labels           0
dtype: int64

In [ ]:
data_valid.isna().sum()

sentence_id      0
words          138
labels           0
dtype: int64

In [ ]:
data_train = data_train.dropna()
data_valid = data_valid.dropna()

In [ ]:
data_train.isna().sum()

sentence_id    0
words          0
labels         0
dtype: int64

In [ ]:
data_valid.isna().sum()

sentence_id    0
words          0
labels         0
dtype: int64

In [ ]:
hist = model.train_model(data_train, eval_data = data_valid, acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/243 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 5 of 10:   0%|          | 0/243 [00:00<?, ?it/s]

Running Epoch 6 of 10:   0%|          | 0/243 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/243 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/243 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/243 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, preds_list = model.eval_model(data_valid)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
## Validation Result
result

{'eval_loss': 0.32299675398552025,
 'precision': 0.65877640203933,
 'recall': 0.7041650447644998,
 'f1_score': 0.6807149576669803}

In [ ]:
result_test, model_outputs_test, preds_list_test = model.eval_model(data_dev)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
## Test Result
result_test

{'eval_loss': 0.30876455921679735,
 'precision': 0.6703910614525139,
 'recall': 0.7100591715976331,
 'f1_score': 0.6896551724137931}

In [ ]:
prediction, model_output = model.predict(["বিএনপির নেতা সাজেদুলের বাসায় এবার জাতিসংঘ প্রতিনিধিদল"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'বিএনপির': 'O'},
  {'নেতা': 'O'},
  {'সাজেদুলের': 'B-Politician'},
  {'বাসায়': 'I-Politician'},
  {'এবার': 'O'},
  {'জাতিসংঘ': 'B-ORG'},
  {'প্রতিনিধিদল': 'O'}]]